<a href="https://colab.research.google.com/github/sera0911/asia_ai_study/blob/main/New_MachinLearning/prj_01_news_category_classification_03_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 전처리 데이터를 모델로 학습시키기

In [ ]:
#모듈 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

In [ ]:
#전처리 된 데이터 불러오기

X_train, X_test, Y_train, Y_test = np.load('/content/datasets/news_data_max_27_size_24151.npy', allow_pickle=True)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(24367, 27)
(2708, 27)
(24367, 6)
(2708, 6)


In [ ]:
# 모델 생성하기

model = Sequential()
#Embedding=라벨링 된 데이터를 원 핫 인코딩 해서 벡터라이징역할(단어들간의 수치적인 관계를 형성시켜준다=단어가 같은 의미들이 같은 방향에 있게끔)
#뒤에는 단어의 총 개수를 주어야한다(=차원의 수)
#output_dim=300,  # 차원 축소 / 차원이 너무 커지면 데이터가 희소해지기 때문에 차원을 300차원으로 제한 (적은 차원에 데이터들을 압축해서 넣음)
model.add(Embedding(24151, 300, input_length=27))  #input_length= 문장의 길이를 줘야한다
model.add(Conv1D(32, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPool1D(pool_size=1))
model.add(LSTM(128, activation='tanh', return_sequences=True))  #순서가 있는 문장이기에 LSTM을 사용해준다, return_sequences=True면 앞에 학습이 모두 출력되고, false면 맨 끝 하나만 출력
model.add(Dropout(0.3))
model.add(LSTM(64, activation='tanh', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64, activation='tanh'))  #return_sequences=디폴트가 false라 LSTM의 마지막 레이어에는 추가안함
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))  #네이버 뉴스 카테고리=6개라서 6

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 27, 300)           7245300   
_________________________________________________________________
conv1d (Conv1D)              (None, 27, 32)            48032     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 27, 32)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 27, 128)           82432     
_________________________________________________________________
dropout (Dropout)            (None, 27, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 27, 64)            49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 27, 64)            0

In [ ]:
# 모델 학습 설정

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_hist = model.fit(X_train, Y_train, batch_size=100, epochs=8, validation_data=(X_test, Y_test))

Epoch 1/8
244/244 [==============================] - 34s 121ms/step - loss: 0.1107 - accuracy: 0.9704 - val_loss: 1.0679 - val_accuracy: 0.7548
Epoch 2/8
244/244 [==============================] - 28s 115ms/step - loss: 0.0845 - accuracy: 0.9765 - val_loss: 1.1014 - val_accuracy: 0.7563
Epoch 3/8
244/244 [==============================] - 28s 115ms/step - loss: 0.0669 - accuracy: 0.9821 - val_loss: 1.2946 - val_accuracy: 0.7470
Epoch 4/8
244/244 [==============================] - 28s 115ms/step - loss: 0.0621 - accuracy: 0.9833 - val_loss: 1.2269 - val_accuracy: 0.7600
Epoch 5/8
244/244 [==============================] - 28s 114ms/step - loss: 0.0532 - accuracy: 0.9863 - val_loss: 1.2896 - val_accuracy: 0.7548
Epoch 6/8
244/244 [==============================] - 28s 115ms/step - loss: 0.0438 - accuracy: 0.9872 - val_loss: 1.4263 - val_accuracy: 0.7585
Epoch 7/8
244/244 [==============================] - 28s 114ms/step - loss: 0.0450 - accuracy: 0.9876 - val_loss: 1.4804 - val_accuracy:

In [ ]:
score = model.evaluate(X_test, Y_test)
print(score[1])  #정확도

85/85 [==============================] - 1s 12ms/step - loss: 1.5509 - accuracy: 0.7312
0.7311668992042542


In [ ]:
# 모델 저장하기

model.save('/content/models/news_classfication_{}.h5'.format(score[1]))